<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Hard-Voting-Edited-Dataset/blob/main/Cross_Project_HV_train_mpxj%2Btest_greenstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [309]:
from numpy import mean
from numpy import std

In [310]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [311]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [312]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [313]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-mpxj 8.0 - mpxj 8.0.csv")


In [314]:
weights['change'] = weights['change'].map({'YES': 1, 'NO': 0})

In [315]:
weights.shape

(821, 8)

In [316]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,1,0,5,29,1,60,5,0
1,0,0,10,3,3,0,0,0
2,1,0,8,43,1,70,8,0
3,2,8,21,792,1,0,5,0
4,13,0,23,292,1,83,44,1


In [317]:
X = weights.drop('change', axis=1)
y = weights['change']

In [318]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [319]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [320]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [321]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.7749077490774908, 0.7564575645756457, 0.7564575645756457]


In [322]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-greenstone 3.0.7 - greenstone 3.0.7.csv")

In [323]:
data['change'] = data['change'].map({'YES': 1, 'NO': 0})

In [324]:
data.shape

(601, 8)

In [325]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,4,0,34,189,1,78,24,1
1,6,0,5,133,1,50,20,1
2,7,0,60,2124,1,0,355,1
3,21,0,15,145,1,85,17,1
4,10,0,13,721,1,91,27,1


In [326]:
X = data.drop('change', axis=1)
y = data['change']

In [327]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [328]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='hard', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [329]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 82.645


In [330]:
print(yhat)

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [331]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[97  3]
 [18  3]]


In [332]:
# predict probabilities
# from sklearn.metrics import roc_auc_score
# pred_prob = ensemble.predict_proba(X_test)
# auc_score = roc_auc_score(y_test, pred_prob[:,1])
# print("Area Under Curve=")
# print(auc_score)

In [333]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.1968856905802946


In [334]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5564285714285714


In [335]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.37225183487986807


In [336]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90       100
           1       0.50      0.14      0.22        21

    accuracy                           0.83       121
   macro avg       0.67      0.56      0.56       121
weighted avg       0.78      0.83      0.78       121

